# Apertus Python Client – Hands-on Examples

This notebook demonstrates how to use the `apertus` client to call the Apertus (Public AI Gateway) inference API. It mirrors familiar OpenAI-style ergonomics and includes sync, streaming, and async flows.

In [1]:
# 1) Import and Auto-Reload
%load_ext autoreload
%autoreload 2

from apertus import Apertus, AsyncApertus, ApertusAPIError
from apertus.types import ChatMessage
import os, json, asyncio, time


In [2]:
# 2) Local Install and Version Check
%pip install -e .

import sys, importlib.metadata as il
print("Python:", sys.version)
print("Apertus version:", il.version("apertus"))

Obtaining file:///home/svenp/Documents/apertus_chat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for apertus (pyproject.toml) ... done
  Created wheel for apertus: filename=apertus-0.1.0-0.editable-py3-none-any.whl size=3674 sha256=fd4115a2f2c93accb25185c6c71e9b47889383ea824e8aceba956b2dc326d178
  Stored in directory: /tmp/pip-ephem-wheel-cache-4b_2uddk/wheels/8a/49/1d/bcff3240cd9a3a7b6851df135623214d5b4300857a9d14f374
Successfully built apertus
  Attempting uninstall: apertus
    Found existing installation: apertus 0.1.0
    Uninstalling apertus-0.1.0:
      Successfully uninstalled apertus-0.1.0
Note: you may need to restart the kernel to use updated packages.
Python: 3.13.0 (main, Oct  7 2024, 05:02:14) [GCC 11.4.0]
Apertus version: 0.1.0


In [5]:
# 3) Quickstart: Core API Call — List Models and pick a model
os.environ.setdefault("APERTUS_API_KEY", "zpka_874ace62ff264105865e9bb92780cde7_79660f8b")  # optionally set here

client = Apertus(api_key=os.getenv("APERTUS_API_KEY"))
models = client.models.list()
print("Available models:", [m.id for m in models.data][:10])

model_id = models.data[0].id if models.data else None
print("Selected model:", model_id)

ValueError: API key is required. Set APERTUS_API_KEY or pass api_key.

In [ ]:
# 4) Non-streaming Chat Completion
assert model_id is not None, "No model available; ensure your API key has access."
resp = client.chat.completions.create(
    model=model_id,
    messages=[{"role": "user", "content": "Hello Apertus!"}],
    temperature=0.2,
    max_tokens=64,
)
print(resp.choices[0].message.content)

In [ ]:
# 5) Streaming Chat Completion
chunks = []
for ev in client.chat.completions.stream(
    model=model_id,
    messages=[{"role": "user", "content": "Stream a short greeting sentence."}],
    temperature=0.2,
    max_tokens=64,
):
    if ev.delta:
        print(ev.delta, end="", flush=True)
        chunks.append(ev.delta)
print()  # newline
streamed_text = "".join(chunks)

In [ ]:
# 6) Async usage
async def async_demo():
    aclient = AsyncApertus()
    models = await aclient.models.list()
    m = models.data[0].id if models.data else None
    assert m, "No model available"
    resp = await aclient.chat.completions.create(
        model=m,
        messages=[{"role": "user", "content": "Async hello!"}],
    )
    print(resp.choices[0].message.content)

await async_demo()

In [ ]:
# 7) Error Handling and Logging
import logging
logging.basicConfig(level=logging.INFO)

try:
    bad = Apertus(api_key="invalid-key")
    bad.models.list()
except ApertusAPIError as e:
    print("Caught ApertusAPIError:", e.status_code, e.message)


In [ ]:
# 8) Configuration via Environment and Parameters
# Using environment variable:
os.environ["APERTUS_API_KEY"] = os.environ.get("APERTUS_API_KEY", "")
client_env = Apertus()

# Override base_url and timeout if you run via a proxy/gateway.
client_custom = Apertus(api_key=os.getenv("APERTUS_API_KEY"), base_url="https://api.publicai.co", timeout=60.0)
print("Configured custom client with longer timeout.")

In [ ]:
# 9) Performance: timeit
%timeit client.chat.completions.create(model=model_id, messages=[{"role": "user", "content": "ping"}], max_tokens=8)

In [ ]:
# 10) Batch Processing with pandas + tqdm
import pandas as pd
from tqdm.auto import tqdm

prompts = pd.DataFrame({"prompt": [
    "Say hi in one word",
    "Say bye in one word",
    "Name a color",
]})

results = []
for row in tqdm(prompts.itertuples(index=False)):
    out = client.chat.completions.create(
        model=model_id,
        messages=[{"role": "user", "content": row.prompt}],
        max_tokens=16,
    )
    results.append(out.choices[0].message.content)

prompts["response"] = results
prompts.head()

In [ ]:
# 11) Visualization
import matplotlib.pyplot as plt

prompts['length'] = prompts['response'].str.len()
prompts.plot(kind='bar', y='length', legend=False, title='Response length by row')
plt.tight_layout()
plt.show()

# 12) Save and Load Artifacts
from pathlib import Path
p = Path('outputs')
p.mkdir(exist_ok=True)
prompts.to_csv(p / 'responses.csv', index=False)
print('Saved to', p / 'responses.csv')

loaded = pd.read_csv(p / 'responses.csv')
loaded.head()

In [ ]:
# 13) Run Unit Tests from Notebook
!pytest -q -k "basic or integration"